# Cumulative event counter

### This notebook counts the cumulative spatio-temporal distribution of cellular events in the local neighbourhood of a series of apoptosis events

1. Load dictionary of apoptoses and their corresponding times to base the spatiotemporal scans around 
2. Load corresponding tracking data
3. Iteratively calculate spatiotemporal distribution of cellular events around apoptoses, saving each distribution out as a .npy distance-time heatmap
4. Compile list of spatiotemporal heatmaps as cumulative heatmap
5. Misc. section: comparing different experimental conditions

In [1]:
import calculate_radial_analysis as calculate
import tools
import render
import dataio

import json
import glob
import os
import re
import numpy as np
from natsort import natsorted
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path

## Set parameters before beginning

In [2]:
focal_cell = 'WT'
radius = 400
t_range = 400
num_bins = 10

## 1) Experiment type dictionary loader
End result of expt_dict that takes the format of :`expt_dict = {expt_ID:expt_type}` 

where `expt_ID` takes the format of: `GV{experiment_ID}_Pos{position_ID}` 

For example: 

```
  apop_dict = {'GV0802/Pos5':'10:90'
                GV0807/Pos8':'50:50'
                GV0813/Pos11':'50:50 Y27632 10 µM'
                                                  }
 ```

In [3]:
if focal_cell == 'Scr':
    cell_type = 'RFP'
if focal_cell == 'WT':
    cell_type = 'GFP '## ie the cell type to base the analysis around, the central cell around which you want to observe the other cell type behaviours
path_to_expt_list = '/home/nathan/data/kraken/h2b/giulia/expt_type.json'
expt_dict = {}
with open(path_to_expt_list) as json_file:
    expt_dict = json.load(json_file)
##filter out non-canonical expts
delete_list =[]
invalid_expts = ['GV0794/Pos3',
 'GV0794/Pos15',
 'GV0798/Pos19',
 'GV0804/Pos3',
 'GV0794/Pos16',
 'GV0812/Pos0',
 'GV0812/Pos1',
 'GV0812/Pos2',
 'GV0812/Pos3',
 'GV0812/Pos4',
 'GV0812/Pos5',]
for i, j in enumerate(expt_dict):
    if expt_dict[j] != '90:10' and expt_dict[j] != '50:50' and expt_dict[j] != '10:90': 
        delete_list.append(j)
    if j in invalid_expts:
        delete_list.append(j)
for i in delete_list:
    try:
        del expt_dict[i]
    except:
        print(i, "not in filtered expt list")
orig_len_expts = len(expt_dict)
expt_dict

GV0812/Pos0 not in filtered expt list
GV0812/Pos1 not in filtered expt list
GV0812/Pos2 not in filtered expt list
GV0812/Pos3 not in filtered expt list
GV0812/Pos4 not in filtered expt list
GV0812/Pos5 not in filtered expt list


{'GV0794/Pos4': '50:50',
 'GV0794/Pos5': '10:90',
 'GV0794/Pos6': '10:90',
 'GV0794/Pos7': '50:50',
 'GV0794/Pos8': '90:10',
 'GV0795/Pos3': '90:10',
 'GV0795/Pos4': '50:50',
 'GV0795/Pos5': '10:90',
 'GV0795/Pos6': '10:90',
 'GV0795/Pos7': '50:50',
 'GV0795/Pos8': '90:10',
 'GV0795/Pos16': '90:10',
 'GV0795/Pos17': '50:50',
 'GV0795/Pos18': '10:90',
 'GV0796/Pos3': '90:10',
 'GV0796/Pos4': '50:50',
 'GV0796/Pos5': '10:90',
 'GV0796/Pos6': '10:90',
 'GV0796/Pos7': '50:50',
 'GV0796/Pos8': '90:10',
 'GV0796/Pos19': '90:10',
 'GV0796/Pos20': '90:10',
 'GV0796/Pos21': '50:50',
 'GV0796/Pos22': '50:50',
 'GV0796/Pos23': '10:90',
 'GV0796/Pos24': '10:90',
 'GV0797/Pos3': '90:10',
 'GV0797/Pos4': '50:50',
 'GV0797/Pos5': '10:90',
 'GV0797/Pos6': '10:90',
 'GV0797/Pos7': '50:50',
 'GV0797/Pos8': '90:10',
 'GV0797/Pos18': '90:10',
 'GV0797/Pos19': '90:10',
 'GV0797/Pos20': '50:50',
 'GV0797/Pos21': '50:50',
 'GV0797/Pos22': '10:90',
 'GV0797/Pos23': '10:90',
 'GV0798/Pos3': '90:10',
 'GV0798/P

In [4]:
### parallelisation 
N_expts = len(expt_dict)
para_frac_min, para_frac_max = int(0*N_expts), int(0.25*N_expts)  ## ie only do first 1/4 of all apops
delete_list =[]
for i, j in enumerate(expt_dict):
    if not para_frac_min <= i< para_frac_max: 
        delete_list.append(j)
for i in delete_list:
    del expt_dict[i]
print("Length of modified dict vs original:", len(expt_dict), "/", orig_len_expts)

Length of modified dict vs original: 18 / 75


## 2) Load corresponding cell tracking data

In [5]:
### load tracking data
path_to_hdf5 = '/home/nathan/data/kraken/h2b/giulia/'
hdf5_file_list = dataio.hdf5_file_finder(path_to_hdf5)

### specify output path and neighbourhood scan parameters
output_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/individual_spatiotemporal_heatmaps/400.400.10/wt_control_wt_div'
Path(output_path).mkdir(parents=True, exist_ok=True)

## 3) Iteratively calculate and save out individual heatmaps

This step can be time consuming (sequential scans around 385 apoptoses took 7 hours on i7)

Function to calculate:

`calculate.iterative_control_heatmap_generator(subject_cells = 'WT', subject_event = 'DIVIDE', expt_dict, N_cells_per_expt, hdf5_file_list, radius, t_range, num_bins, output_path)`

Where the necessary input variables are as follows:
- `focal_cells` = The population of cells to use as the focal point of the scan, 'WT'/'Scr'/'All'
- `subject_cells` = The population of cells to scan over looking for events, 'WT'/'Scr'/'All'
- `subject_event` = The event type to look for, 'DIVIDE' or 'APOPTOSIS'
- `expt_dict` = Previously defined python dictionary of {experiment ID: experiment type}
- `N_cells_per_expt` = The number of cells and corresponding heatmaps to scan/generate per experiment
- `hdf5_file_list` = List of locations of tracking files
- `radius` = Distance to scan over around apoptosis
- `t_range` = Time range to scan over around apoptosis (will scan t_range/2 before and after apoptosis)
- `num_bins` = Number of bins the histogram is sorted into
- `output_path` = Empty directory to save out individual apoptosis heatmap .npy arrays into

Note: the total number of cells scanned over, and subsequent number of heatmaps generated, will be `N_cells_per_expt` * `len(expt_dict)`

In [6]:
N_cells_per_expt = 25
print("Total cells to be scanned over/heatmaps generated:", N_cells_per_expt*len(expt_dict))

Total cells to be scanned over/heatmaps generated: 450


In [7]:
N_cells, error_log, success_log = calculate.iterative_control_heatmap_generator(
    'WT',
    'WT', ## 'Scr',
    'DIVIDE', ##random
    expt_dict, 
    N_cells_per_expt,
    hdf5_file_list, 
    radius, 
    t_range, 
    num_bins, 
    output_path)

  0%|          | 0/18 [00:00<?, ?it/s][INFO][2021/05/10 04:01:58 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5...
[INFO][2021/05/10 04:01:58 PM] Loading tracks/obj_type_1


Loading GV0794/Pos4/
Btrack version no.: 0.4.0


[INFO][2021/05/10 04:01:58 PM] Loading objects/obj_type_1 (69360, 5) (64986 filtered: area>=100)
[INFO][2021/05/10 04:01:59 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5
[INFO][2021/05/10 04:01:59 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5...
[INFO][2021/05/10 04:01:59 PM] Loading tracks/obj_type_2
[INFO][2021/05/10 04:01:59 PM] Loading objects/obj_type_2 (13865, 5) (10538 filtered: area>=100)
[INFO][2021/05/10 04:01:59 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0794/Pos4/


  6%|▌         | 1/18 [06:38<1:52:46, 398.02s/it][INFO][2021/05/10 04:08:36 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos5/Pos5_aligned/HDF/segmented.hdf5...
[INFO][2021/05/10 04:08:36 PM] Loading tracks/obj_type_1
[INFO][2021/05/10 04:08:36 PM] Loading objects/obj_type_1 (31315, 5) (20377 filtered: area>=100)


...

 94%|█████████▍| 17/18 [6:57:14<31:11, 1871.23s/it]  [INFO][2021/05/10 10:59:13 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0796/Pos6/Pos6_aligned/HDF/segmented.hdf5...
[INFO][2021/05/10 10:59:13 PM] Loading tracks/obj_type_1


Loading GV0796/Pos6/
Btrack version no.: 0.4.0


[INFO][2021/05/10 10:59:13 PM] Loading objects/obj_type_1 (70354, 5) (57413 filtered: area>=100)
[INFO][2021/05/10 10:59:14 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0796/Pos6/Pos6_aligned/HDF/segmented.hdf5
[INFO][2021/05/10 10:59:14 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0796/Pos6/Pos6_aligned/HDF/segmented.hdf5...
[INFO][2021/05/10 10:59:14 PM] Loading tracks/obj_type_2
[INFO][2021/05/10 10:59:14 PM] Loading objects/obj_type_2 (88276, 5) (68435 filtered: area>=100)
[INFO][2021/05/10 10:59:15 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0796/Pos6/Pos6_aligned/HDF/segmented.hdf5


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0796/Pos6/


100%|██████████| 18/18 [7:05:06<00:00, 1417.04s/it]


In [8]:
N_cells ## scope of analysis (number of apoptoses scanned around)

450

In [9]:
len(error_log)

0

In [10]:
error_log, success_log

([],
 ['/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/individual_spatiotemporal_heatmaps/400.400.10/wt_control_wt_div/GV0794_Pos4_ID:349_t:713_N_cells / /home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/individual_spatiotemporal_heatmaps/400.400.10/wt_control_wt_div/GV0794_Pos4_ID:349_t:713_N_events saved out successfully',
  '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/individual_spatiotemporal_heatmaps/400.400.10/wt_control_wt_div/GV0794_Pos4_ID:498_t:914_N_cells / /home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/individual_spatiotemporal_heatmaps/400.400.10/wt_control_wt_div/GV0794_Pos4_ID:498_t:914_N_events saved out successfully',
  '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/individual_spatiotemporal_heatmaps/400.400.10/wt_control_wt_div/GV0794_Pos4_ID:348_t:796_N_cells / /home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/individual_spatiotemporal_heatmaps/400.400.10/w

# debug - checking if i have all experiment data type

In [11]:
expt_dict

{'GV0794/Pos4': '50:50',
 'GV0794/Pos5': '10:90',
 'GV0794/Pos6': '10:90',
 'GV0794/Pos7': '50:50',
 'GV0794/Pos8': '90:10',
 'GV0795/Pos3': '90:10',
 'GV0795/Pos4': '50:50',
 'GV0795/Pos5': '10:90',
 'GV0795/Pos6': '10:90',
 'GV0795/Pos7': '50:50',
 'GV0795/Pos8': '90:10',
 'GV0795/Pos16': '90:10',
 'GV0795/Pos17': '50:50',
 'GV0795/Pos18': '10:90',
 'GV0796/Pos3': '90:10',
 'GV0796/Pos4': '50:50',
 'GV0796/Pos5': '10:90',
 'GV0796/Pos6': '10:90'}

## 4) Compile individual heatmaps into cumulative plots 

Compile cumulative heatmap by iteratively adding single-apoptosis heatmaps from previous output or different pre-established output 

In [12]:
#single_cell_heatmap_location = output_path
single_cell_heatmap_location = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/individual_spatiotemporal_heatmaps/800,20/wt_apop_scr_div'
save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/wt_apop_scr_div'
Path(save_path).mkdir(parents=True, exist_ok=True)
radius, t_range, num_bins = 800, 800, 20#400, 400, 10

In [ ]:
N_cells, N_events = np.zeros((num_bins, num_bins)), np.zeros((num_bins, num_bins))
N = 0
for heatmap_ID in natsorted(os.listdir(single_cell_heatmap_location)):
    if 'N_cells' in heatmap_ID:
        N_cells += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
    if 'N_events' in heatmap_ID:
        N_events += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
        N+=1
N

In [ ]:
print("The number of scr. cells counted is equal to = ", N_cells.sum())

Function to plot heatmaps: 

`plot_cumulative(input_2d_hist, num_bins, radius, t_range, title, expt_label, cb_label, save_path, SI = True)`

Where the variables needed are as follows:

- `input_2d_hist` = Input 2D histogram
- `num_bins` = Number of bins the histogram is sorted into
- `radius` = Spatial extent of scan
- `t_range` = Temporal range of scan
- `title` = Title/save out filename of heatmap
- `label` = Optional label for heatmap
- `cb_label` = Label for colourbar
- `save_path` = Ouput save path, if empty string then plot will not be saved out
- `SI` = If True then plots the xy in SI base unit instead of pixel/frames

In [ ]:
render.plot_cumulative(input_2d_hist = N_cells,
                num_bins  = 20,
                radius  = 800,
                t_range = 800, 
                title = 'Spatiotemporal dist. of Scribble cells \n around wild-type apoptoses (N={})'.format(N), 
                label = '', 
                cb_label = 'Number of cell apperances', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/wt_apop_scr_div',
                SI = True) ## SI is standard 

In [ ]:
render.plot_cumulative(input_2d_hist = N_events,
                num_bins  = 20,
                radius  = 800,
                t_range = 800, 
                title = 'Spatiotemporal dist. of Scribble mitoses \n around wild-type apoptoses (N={})'.format(N), 
                label = '', 
                cb_label = 'Number of Scribble divisions', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/wt_apop_scr_div',
                SI = True)

In [ ]:
render.plot_cumulative(input_2d_hist = N_events/N_cells,
                num_bins  = 20,
                radius  = 800,
                t_range = 800, 
                title = 'Spatiotemporal dist. of probability of Scribble division \n around wild-type apoptoses (N={})'.format(N), 
                label = '', 
                cb_label = 'Probability of division', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/wt_apop_scr_div',
                SI = True)

# 5) Miscellaneous section: segregating cumulative plots based on experiment type

50:50 expt

In [ ]:
single_cell_heatmap_location = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/individual_spatiotemporal_heatmaps/800,20/scr_apop_wt_div'
save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/scr_apop_wt_div/expt_types'
Path(save_path).mkdir(parents=True, exist_ok=True)

In [ ]:
expt_list_5050 = ['GV0794/Pos4',
                 'GV0794/Pos7',
                 'GV0794/Pos16',
                 'GV0795/Pos4',
                 'GV0795/Pos7',
                 'GV0795/Pos17',
                 'GV0796/Pos4',
                 'GV0796/Pos7',
                 'GV0796/Pos21',
                 'GV0796/Pos22',
                 'GV0797/Pos4',
                 'GV0797/Pos7',
                 'GV0797/Pos20',
                 'GV0797/Pos21',
                 'GV0798/Pos4',
                 'GV0798/Pos7',
                 'GV0798/Pos20',
                 'GV0798/Pos21',
                 'GV0800/Pos4', 
                 'GV0800/Pos7',
                 'GV0800/Pos16',
                 'GV0801/Pos4',
                 'GV0802/Pos4',
                 'GV0804/Pos4',
                 'GV0805/Pos4',
                 'GV0806/Pos4',
                 'GV0807/Pos4',
                 'GV0812/Pos0',
                 'GV0812/Pos1',
                 'GV0812/Pos2',
                 'GV0812/Pos3',
                 'GV0812/Pos4',
                 'GV0812/Pos5',
                 'GV0813/Pos0',
                 'GV0813/Pos1',
                 'GV0813/Pos2',
                 'GV0813/Pos3',
                 'GV0813/Pos4',
                 'GV0813/Pos5',
                 'GV0814/Pos0',
                 'GV0814/Pos11',
                 'GV0814/Pos12',
                 'GV0814/Pos23',
                 'GV0815/Pos0',
                 'GV0815/Pos11',
                 'GV0815/Pos12',
                 'GV0815/Pos23',
                 'GV0816/Pos0',
                 'GV0816/Pos13',
                 'GV0816/Pos14',
                 'GV0816/Pos23',
                 'GV0817/Pos0',
                 'GV0817/Pos11',
                 'GV0817/Pos12',
                 'GV0817/Pos23',
                 'GV0818/Pos12',
                 'GV0818/Pos13',
                 'GV0819/Pos0',
                 'GV0819/Pos1',
                 'GV0819/Pos2',
                 'GV0819/Pos3',
                 'GV0819/Pos4',
                 'GV0819/Pos5',
                 'GV0820/Pos0',
                 'GV0820/Pos1',
                 'GV0820/Pos2',
                 'GV0820/Pos3',
                 'GV0820/Pos4',
                 'GV0820/Pos5']

In [ ]:
N_cells, N_events = np.zeros((num_bins, num_bins)), np.zeros((num_bins, num_bins))
N = 0
for heatmap_ID in natsorted(os.listdir(single_cell_heatmap_location)):
    #print(heatmap_ID.split('_P'))
    expt = 'GV' +str(re.findall(r"GV(\d+)", heatmap_ID)[0])
    position = re.findall(r"Pos(\d+)", heatmap_ID)[0]
    if os.path.join(expt,'Pos'+position) in expt_list_5050:
        if 'N_cells' in heatmap_ID:
            N_cells += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
        if 'N_events' in heatmap_ID:
            N_events += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
            N+=1
N

In [ ]:
render.plot_cumulative(input_2d_hist = N_cells,
                num_bins  = 20,
                radius  = 800,
                t_range = 800, 
                title = 'Spatiotemporal dist. of wild-type cells \n around Scribble apoptoses (N={}, 50:50 wt:scr expts only)'.format(N), 
                label = '', 
                cb_label = 'Number of cell apperances', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/scr_apop_wt_div/expt_types',
                SI = True) ## SI is standard 

In [ ]:
render.plot_cumulative(input_2d_hist = N_events,
                num_bins  = 20,
                radius  = 800,
                t_range = 800, 
                title = 'Spatiotemporal dist. of wild-type division \n around Scribble apoptoses (N={}, 50:50 wt:scr expts only)'.format(N), 
                label = '', 
                cb_label = 'Number of wild-type divisions', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/scr_apop_wt_div/expt_types',
                SI = True)

In [ ]:
render.plot_cumulative(input_2d_hist = N_events/N_cells,
                num_bins  = 20,
                radius  = 800,
                t_range = 800,  
                title = 'Spatiotemporal dist. of probability of Scribble division \n around wild-type apoptoses (N={}, 50:50 wt:scr expts only)'.format(N), 
                label = '', 
                cb_label = 'Probability of division', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/scr_apop_wt_div/expt_types',
                SI = True)

90:10 wt:scr

In [ ]:
expt_list_9010 = ['GV0794/Pos3',
 'GV0794/Pos8',
 'GV0794/Pos15',
 'GV0795/Pos3',
 'GV0795/Pos8',
 'GV0795/Pos16',
 'GV0796/Pos3',
 'GV0796/Pos8',
 'GV0796/Pos19',
 'GV0796/Pos20',
 'GV0797/Pos3',
 'GV0797/Pos8',
 'GV0797/Pos18',
 'GV0797/Pos19',
 'GV0798/Pos3',
 'GV0798/Pos8',
 'GV0798/Pos18',
 'GV0798/Pos19',
 'GV0800/Pos3',
 'GV0800/Pos8',
 'GV0800/Pos15',
 'GV0801/Pos3',
 'GV0802/Pos3',
 'GV0804/Pos3',
 'GV0805/Pos3',
 'GV0806/Pos3',
 'GV0807/Pos3']

In [ ]:
N_cells, N_events = np.zeros((num_bins, num_bins)), np.zeros((num_bins, num_bins))
N=0
for heatmap_ID in natsorted(os.listdir(single_cell_heatmap_location)):
    #print(heatmap_ID.split('_P'))
    expt = 'GV' +str(re.findall(r"GV(\d+)", heatmap_ID)[0])
    position = re.findall(r"Pos(\d+)", heatmap_ID)[0]
    if os.path.join(expt,'Pos'+position) in expt_list_9010:
        if 'N_cells' in heatmap_ID:
            N_cells += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
        if 'N_events' in heatmap_ID:
            N_events += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
            N+=1
N

In [ ]:
render.plot_cumulative(input_2d_hist = N_cells,
                num_bins  = 20,
                radius  = 800,
                t_range = 800, 
                title = 'Spatiotemporal dist. of wild-type cells \n around Scribble apoptoses (N={}, 90:10 wt:scr expts only)'.format(N), 
                label = '', 
                cb_label = 'Number of cell apperances', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/scr_apop_wt_div/expt_types',
                SI = True) ## SI is standard 

In [ ]:
render.plot_cumulative(input_2d_hist = N_events,
                num_bins  = 20,
                radius  = 800,
                t_range = 800, 
                title = 'Spatiotemporal dist. of wild-type division \n around Scribble apoptoses (N={}, 90:10 wt:scr expts only)'.format(N),
                label = '', 
                cb_label = 'Number of wild-type divisions', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/scr_apop_wt_div/expt_types',
                SI = True)

In [ ]:
render.plot_cumulative(input_2d_hist = N_events/N_cells,
                num_bins  = 20,
                radius  = 800,
                t_range = 800, 
                title = 'Spatiotemporal dist. of probability of wild-type division \n around Scribble apoptoses (N={}, 90:10 wt:scr expts only)'.format(N),
                label = '', 
                cb_label = 'Probability of division', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/scr_apop_wt_div/expt_types',
                SI = True)

10:90 wt:scr

In [ ]:
expt_list_1090 = ['GV0794/Pos5',
 'GV0794/Pos6',
 'GV0795/Pos5',
 'GV0795/Pos6',
 'GV0795/Pos18',
 'GV0796/Pos5',
 'GV0796/Pos6',
 'GV0796/Pos23',
 'GV0796/Pos24',
 'GV0797/Pos5',
 'GV0797/Pos6',
 'GV0797/Pos22',
 'GV0797/Pos23',
 'GV0798/Pos5',
 'GV0798/Pos6',
 'GV0798/Pos22',
 'GV0798/Pos23',
 'GV0800/Pos5',
 'GV0800/Pos6',
 'GV0800/Pos17',
 'GV0801/Pos5',
 'GV0802/Pos5',
 'GV0804/Pos5',
 'GV0805/Pos5',
 'GV0806/Pos5',
 'GV0807/Pos5']

In [ ]:
N_cells, N_events = np.zeros((num_bins, num_bins)), np.zeros((num_bins, num_bins))
N=0
for heatmap_ID in natsorted(os.listdir(single_cell_heatmap_location)):
    #print(heatmap_ID.split('_P'))
    expt = 'GV' +str(re.findall(r"GV(\d+)", heatmap_ID)[0])
    position = re.findall(r"Pos(\d+)", heatmap_ID)[0]
    if os.path.join(expt,'Pos'+position) in expt_list_1090:
        if 'N_cells' in heatmap_ID:
            N_cells += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
        if 'N_events' in heatmap_ID:
            N_events += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
            N+=1
N

In [ ]:
render.plot_cumulative(input_2d_hist = N_cells,
                num_bins  = 20,
                radius  = 800,
                t_range = 800, 
                title = 'Spatiotemporal dist. of wild-type cells around \nScribble apoptoses (N={}, 10:90 wt:scr expts only)'.format(N), 
                label = '', 
                cb_label = 'Number of cell apperances', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/scr_apop_wt_div/expt_types',
                SI = True) ## SI is standard 

In [ ]:
render.plot_cumulative(input_2d_hist = N_events,
                num_bins  = 20,
                radius  = 800,
                t_range = 800,  
                title = 'Spatiotemporal dist. of wild-type division around \nScribble apoptoses (N={}, 10:90 wt:scr expts only)'.format(N),
                label = '', 
                cb_label = 'Number of wild-type divisions', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/scr_apop_wt_div/expt_types',
                SI = True)

In [ ]:
render.plot_cumulative(input_2d_hist = N_events/N_cells,
                num_bins  = 20,
                radius  = 800,
                t_range = 800, 
                title = 'Spatiotemporal dist. of probability of wild-type division around \nScribble apoptoses (N={}, 10:90 wt:scr expts only)'.format(N),
                label = '', 
                cb_label = 'Probability of division', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/800,20/scr_apop_wt_div/expt_types',
                SI = True)

Write a function to do a time evolving plot that iteratively adds single cell heat maps according to the apoptosis time?